<!--associate:
# The comment must be at the very beginning of a cell, by itself, starting with 'associate:'. 
# Since it is not meant to appear in the output when run, the assumption is that it can 
# require a cell to itself. This refers to  directories, relative to the notebook.
TAI.png
-->

# Module 1: Is the hourglass model for gene expression really supported by the data?
### Paper to be examined: 
“A phylogenetically based transcriptome age index mirrors ontogenetic divergence patterns”, Nature 9;468(7325):815-8 (2010)[1]
### Key claim of the paper: 
"Gene expression follows the so-called hourglass pattern observed for morphological features of development, which are most similar to each other in the phylotypic stage in mid-development."

### Schedule:
* H1: General introduction to the paper/motivation
* H2-3: Write code to import the data and start computing transcriptome age index (TAI)
* H4-6: Aim to reproduce figure 1 of the paper – help/scripts will be given if needed.
* H7: Discussion: “Are you convinced of this result? What might have gone wrong?”
* H8: Redo analysis using log-transformed data
* H9: Summarize results (e.g. on this wiki)


### Key bioinformatics concept of this module: 
"Data normalization is important and can impact the results of subsequent analyses!"


# Installation and Setup

* Install the Anaconda distribution of Python 3.x.


# Libraries
Will be using [**GEOparse**](https://geoparse.readthedocs.io/en/latest/usage.html#working-with-geo-accession) for for fetching gene expression data and [**pandas**](https://pandas.pydata.org/pandas-docs/stable/10min.html) for data manipulation and preprocessing.

In [ ]:
########### import necessary packages
import pandas as pd
import numpy as np
import GEOparse
import matplotlib.pyplot as plt

# Read gene expression data 


In [ ]:
############# Download the data
file_name = 'GSE24616'
# write your code here 
gse = GEOparse.get_GEO(geo=file_name, destdir="./")

## GSE data structure:
Let's take a look at gene expression data stucture and accession.

**Data Sturcture:**
    - gse.gsms
        - gse.gsms.metadata
        - gse.gsms.name
        - gse.gsms.table
    - gse.gpl
        - gse.gpl.metadata
        - gse.gpl.name
        - gse.gpl.table
        
**GSE file name:** GSE24616

In [ ]:
########### Explore an example of GSE content
print ("GSM example:")
for gsm_name, gsm in gse.gsms.items():
    print ("Name: ", gsm_name)
    print ('*'*100)
    print ("Metadata:"),
    for key, value in gsm.metadata.items():
        print(" - %s : %s" % (key, ", ".join(value)))
    print('*'*100)
    print ("Table data:"),
    print (gsm.table.head())
    break

In [ ]:
print ("GPL example:")
for gpl_name, gpl in gse.gpls.items():
    print ("Name: ", gpl_name)
    print ('*'*100)
    print ("Metadata:"),
    for key, value in gpl.metadata.items():
        print (" - %s : %s" % (key, ", ".join(value)))
    print ('*'*100)
    print ("Table data:"),
    print (gpl.table.head())
    break

# Read age index data file

In [ ]:
########### Read in age index data
age_index = pd.read_csv('danio_age_index.txt', sep='\t', header=None)
age_index.columns = ["GeneID","ProbeID","age"]
########### Set ProbeID as the index of dataframe
age_index.set_index('ProbeID',inplace=True)
age_index.head()

# Pre processing gene expression data:
Gene expression data needs to be extracted from GSE data structure. Preprocessing steps are:
1. Extract the metadata
2. Extract the gene expression data
3. Add age index data to the gene expression data
4. Get average for the genes with multiple probesets
5. Select mixed and female samples 
6. Get the average gene expression for similar time points¶

## 1) Extract  metadata of samples in gene expression data
Complementary information about the samples is stores in  **gsm.metadata** including sex, developmental stage and the sample name. A sample metadata looks like:

"characteristics_ch1 : strain: wild type, developmental stage: adult, developmental timing: 1y9m, gender: mixed, number of individuals per sample: 2"

All these infomations are stored in an string and we need to extract them by some String Formatting Operations.

In [ ]:
############### Extract GSE metadata
characteristics = {"stage":[],"time":[],"sex":[],"sample_name":[]}
for gsm_name, gsm in sorted(gse.gsms.items()): # gsm: metadata, columns, table
    characteristics["stage"].append(gsm.metadata['characteristics_ch1'][1].split(":")[1].strip())
    characteristics["time"].append(gsm.metadata['characteristics_ch1'][2].split(":")[1].strip())
    characteristics["sex"].append(gsm.metadata['characteristics_ch1'][3].split(":")[1].strip())
    characteristics["sample_name"].append(gsm_name)
char_df = pd.DataFrame(characteristics,index = characteristics["sample_name"])
char_df.head()

## 2) Extract the gene expression data

In [ ]:
############### Extract the gene expression data
data = gse.pivot_samples('VALUE') 
############# Add ProbeID as the index of gene expression dataframe
gpl = list(gse.gpls.values())[0]
data.set_index(gpl.table.SPOT_ID,inplace=True)
############ Let's look at the gene expression data
data.head()
plt.imshow(np.log(data.values),aspect='auto')
plt.show()

## 3) Add age index data to the gene expression data

In [ ]:
matched_data =data.join(age_index,how='inner').groupby(level=0).last()
print("Age index data dimensios: ",age_index.shape)
print ("Gene expression data dimensions: ", data.shape)
print ("Number of genes with age index: ",matched_data.shape)


## 4) Get average for the genes with multiple probesets

In [ ]:
########### Average out multiple transcripts
unique_data = matched_data.groupby('GeneID').mean()
print ("Number of unique transcripts: ",unique_data.shape[0])

## 5) Select mixed and female samples

In [ ]:
char_df= char_df[char_df["sex"]!="male"]

## 6) Get the average gene expression for similar time points

In [ ]:
########## find samples of the same time points

experiment_index=[]
time_stamps = char_df.time.unique()
for t in time_stamps:
    experiment_index.append(char_df[char_df['time']==t].index.tolist())
########### average the samples for similar time points
set_mean = {}
stages = []
timestamps=[]
for d in range(len(experiment_index)) :
    sample_list = experiment_index[d]
    set_mean[d] = unique_data[sample_list].mean(axis=1)
    stages.append(char_df[char_df.index.isin(sample_list)].stage[0])
    timestamps.append(char_df[char_df.index.isin(sample_list)].time[0])
mean_data = pd.DataFrame(set_mean )
mean_data.columns= stages
mean_data.head()

# TAI Calculation

In [ ]:
#### Calculating TAI  with looping over developmental stage
TAI=[]
for i in range(expression_data.shape[1]):
    TAI_i=0
    for g in range(expression_data.shape[0]):
        TAI_i+=age_indices.values[g]* expression_data[g,i]
    TAI.append(TAI_i/sum(expression_data[:,i]))

unique_stages= list(set(stages))
color_list = plt.cm.tab10(np.linspace(0, 1, len(unique_stages))) 
color = {unique_stages[i]:color_list[i] for i in range(len(unique_stages))}
my_col= [color[i] for i in stages]

plt.figure(figsize=(20,10))
plt.scatter(range(len(TAI)),TAI,color=my_col,linestyle='-')
plt.xticks(range(len(TAI)),timestamps,rotation=45,size=12)
plt.ylabel("Transcriptome Age Index",size=30)
plt.xlabel("Developmental timing",size=30)
markers = [plt.Line2D([0,0],[0,0],color=c, marker='o', linestyle='') for c in color.values()]
plt.legend(markers,color.keys(),loc='upper center')
plt.savefig('TAI.png')
plt.show()

In [ ]:
########### Calculating TAI with matrix maltiplication
age_indices =unique_data['age']
expression_data = mean_data.values
product = np.dot(expression_data.T,age_indices)
mean_expression = expression_data.T.sum(1)
TAI = np.divide(product,mean_expression)


#### define color map

unique_stages= list(set(stages))
color_list = plt.cm.tab10(np.linspace(0, 1, len(unique_stages))) 
color = {unique_stages[i]:color_list[i] for i in range(len(unique_stages))}
my_col= [color[i] for i in stages]

plt.figure(figsize=(20,10))
plt.scatter(range(len(TAI)),TAI,color=my_col,linestyle='-')
plt.xticks(range(len(TAI)),timestamps,rotation=45,size=12)
plt.ylabel("Transcriptome Age Index",size=30)
plt.xlabel("Developmental timing",size=30)
markers = [plt.Line2D([0,0],[0,0],color=c, marker='o', linestyle='') for c in color.values()]
plt.legend(markers,color.keys(),loc='upper center')
plt.savefig('TAI.png')
plt.show()


# Save the pre-processed expression data to a file

In [ ]:
########### add age index to expression and write to file
mean_data.join(unique_data['age']).reset_index().to_csv("ProcessedMicroarrayData_py.txt",sep='\t',index=False)

## Plot histogram of gene expression data vs log values

In [ ]:
plt.figure()
plt.hist(expression_data.reshape((expression_data.shape[0]*expression_data.shape[1],1)),bins=100)
plt.show()

plt.figure()
plt.hist(np.log(expression_data.reshape((expression_data.shape[0]*expression_data.shape[1],1))),bins=100)
plt.show()


## TAI calculation with log normalization of gene expression

In [ ]:
########### Calculating TAI


age_indices =unique_data['age']
expression_data = np.log(mean_data.values)
product = np.dot(expression_data.T,age_indices)
mean_expression = expression_data.T.sum(1)
TAI = np.divide(product,mean_expression)

#### define color map

unique_stages= list(set(stages))
color_list = plt.cm.tab10(np.linspace(0, 1, len(unique_stages))) 
color = {unique_stages[i]:color_list[i] for i in range(len(unique_stages))}
my_col= [color[i] for i in stages]

plt.figure(figsize=(20,10))
plt.scatter(range(len(TAI)),TAI,color=my_col,linestyle='-')
plt.xticks(range(len(TAI)),timestamps,rotation=45,size=12)
plt.ylabel("Transcriptome Age Index",size=30)
plt.xlabel("Developmental timing",size=30)
markers = [plt.Line2D([0,0],[0,0],color=c, marker='o', linestyle='') for c in color.values()]
plt.legend(markers,color.keys(),loc='upper center')
plt.savefig('TAI.png')
plt.show()


## TAI calculation with z-score normalization of gene expression


In [ ]:
########### Calculating TAI


age_indices =unique_data['age']
expression_data = (mean_data.apply(lambda x:(x-(sum(x)/len(x)))/np.std(x))).values
print(expression_data)
product = np.dot(expression_data.T,age_indices)
mean_expression = expression_data.T.sum(1)
TAI = np.divide(product,mean_expression)

#### define color map

unique_stages= list(set(stages))
color_list = plt.cm.tab10(np.linspace(0, 1, len(unique_stages))) 
color = {unique_stages[i]:color_list[i] for i in range(len(unique_stages))}
my_col= [color[i] for i in stages]

plt.figure(figsize=(20,10))
plt.scatter(range(len(TAI)),TAI,color=my_col,linestyle='-')
plt.xticks(range(len(TAI)),timestamps,rotation=45,size=12)
plt.ylabel("Transcriptome Age Index",size=30)
plt.xlabel("Developmental timing",size=30)
markers = [plt.Line2D([0,0],[0,0],color=c, marker='o', linestyle='') for c in color.values()]
plt.legend(markers,color.keys(),loc='upper center')
plt.savefig('TAI.png')
plt.show()

plt.figure()
plt.hist(expression_data.reshape((expression_data.shape[0]*expression_data.shape[1],1)),bins=1000)
plt.show()


## TAI calculation with absolute  normalization of gene expression


In [ ]:
age_indices =unique_data['age']
expression_data  = (mean_data.apply(lambda x:np.abs(x-(sum(x)/len(x)))/np.std(x),axis=0)).values
print(expression_data.shape)
product = np.dot(expression_data.T,age_indices)
mean_expression = expression_data.T.sum(1)
TAI = np.divide(product,mean_expression)

#### define color map

unique_stages= list(set(stages))
color_list = plt.cm.tab10(np.linspace(0, 1, len(unique_stages))) 
color = {unique_stages[i]:color_list[i] for i in range(len(unique_stages))}
my_col= [color[i] for i in stages]

plt.figure(figsize=(20,10))
plt.scatter(range(len(TAI)),TAI,color=my_col,linestyle='-')
plt.xticks(range(len(TAI)),timestamps,rotation=45,size=12)
plt.ylabel("Transcriptome Age Index",size=30)
plt.xlabel("Developmental timing",size=30)
markers = [plt.Line2D([0,0],[0,0],color=c, marker='o', linestyle='') for c in color.values()]
plt.legend(markers,color.keys(),loc='upper center')
plt.savefig('TAI.png')
plt.show()

plt.figure()
plt.hist(expression_data.reshape((expression_data.shape[0]*expression_data.shape[1],1)),bins=100)
plt.show()